In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import glob 
import os

import re, nltk, spacy, string

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS



In [10]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' %re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

nlp = spacy.load('en') 

def lemmatizer(text):
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [11]:
def column_cleaner(df, header):
    df_clean = pd.DataFrame(df[header].apply(lambda x: clean_text(x)))
    df_clean['{}_lemmatize'.format(header)] = df_clean.apply(lambda x: lemmatizer(x[header]), axis=1)
    df_clean['{}_clean'.format(header)] = df_clean['{}_lemmatize'.format(header)].str.replace('-PRON-', '')
    return df_clean

In [12]:
def distributionOfCharLength(df_clean, header):
    plt.figure(figsize=(10,6))
    doc_lens = [len(d) for d in df_clean[header]]
    plt.hist(doc_lens, bins = 30)
    plt.title('Distribution of {} character length'.format(header))
    plt.ylabel('Number of {}s'.format(header))
    plt.xlabel('{} length'.format(header))
    sns.despine();

In [13]:
def top_words_visual(df_clean, header):
    mpl.rcParams['figure.figsize']=(12.0,12.0)  
    mpl.rcParams['font.size']=12            
    mpl.rcParams['savefig.dpi']=100             
    mpl.rcParams['figure.subplot.bottom']=.1 
    stopwords = set(STOPWORDS)

    wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=500,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(df_clean[header]))

    print(wordcloud)
    fig = plt.figure(1)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show();

In [32]:
customer_success = pd.read_csv("Data/All/CustomerSuccess.csv")

In [35]:
customer_success['prev_title_1'].value_counts()

Customer Success Manager                                                             16
Customer Success Advocate                                                             5
Customer Success                                                                      4
Customer Success Specialist                                                           3
Senior Customer Success Manager                                                       3
                                                                                     ..
Analytics Platform Services Reference Manager, IBM Analtyics, Cloud Data Services     1
Director Customer Experience                                                          1
Technical Support Engineer                                                            1
Client Relations Manager                                                              1
Customer Success Advocate - Cash App                                                  1
Name: prev_title_1, Length: 312,

In [37]:
df_clean = pd.DataFrame(customer_success[customer_success['prev_title_1'].notna()]['prev_title_1'].apply(lambda x: clean_text(x)))
df_clean["prev_title_1_lemmatize"] =  df_clean.apply(lambda x: lemmatizer(x['prev_title_1']), axis=1)
df_clean['prev_title_1_clean'] = df_clean['prev_title_1_lemmatize'].str.replace('-PRON-', '')


In [38]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df_clean['prev_title_1_clean'], 30)
df2 = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

fig = go.Figure([go.Bar(x=df2['unigram'], y=df2['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 unigrams in the question text after removing stop words and lemmatization"))



In [39]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df_clean['prev_title_1_clean'], 20)
df3 = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

fig = go.Figure([go.Bar(x=df3['bigram'], y=df3['count'])])
fig.update_layout(title=go.layout.Title(text="Top 20 bigrams text after removing stop words and lemmatization"))
fig.show()

In [40]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df_clean['prev_title_1'], 20)
df4 = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

fig = go.Figure([go.Bar(x=df4['trigram'], y=df4['count'])])
fig.update_layout(title=go.layout.Title(text="Top 20 trigrams in the question text"))
fig.show()

## Description

In [41]:
df_clean['']

,prev_title_1,prev_title_1_lemmatize,prev_title_1_clean
0,customer engagement director,customer engagement director,customer engagement director
1,ibm senior project manager,ibm senior project manager,ibm senior project manager
2,tivoli software sales manager west region,tivoli software sale manager west region,tivoli software sale manager west region
3,manager watson expert cognitive architects,manager watson expert cognitive architect,manager watson expert cognitive architect
4,founder managing director,founder manage director,founder manage director
...,...,...,...
343,global head customer success,global head customer success,global head customer success
344,customer success manager global,customer success manager global,customer success manager global
345,head of partner success south india ubereats,head of partner success south india ubereats,head of partner success south india ubereats
346,specialist public and social sector practice,specialist public and social sector practice,specialist public and social sector practice


In [50]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                       
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=5000,          
                            )

data_vectorized = vectorizer.fit_transform(df_clean_1['prev_title_1'])

lda_model = LatentDirichletAllocation(n_components=10, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      32.420952   8.115230       1        1  25.873158
4       0.648955  71.707100       2        1  14.290231
9      71.019684   9.971006       3        1  11.102167
8       3.785981  33.185135       4        1  10.486612
7     -42.666500  -9.222070       5        1  10.230241
3      -3.621689 -43.526516       6        1   7.043978
0      -3.632402  -4.171708       7        1   6.684667
6     -36.584400  35.804413       8        1   5.186277
2      44.391380  50.615383       9        1   5.020924
5      40.235809 -31.591665      10        1   4.081745, topic_info=          Term       Freq      Total Category  logprob  loglift
17    customer  68.000000  68.000000  Default  30.0000  30.0000
78     success  59.000000  59.000000  Default  29.0000  29.0000
41     manager  92.000000  92.000000  Default  28.0000  28.0000
15  consultant  18.000000  18.000000  Default  27.0000  27.0000
73  specialist  12.000000  12.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
56     product   0.170896   8.300917  Topic10  -5.3749  -0.6844
32        head   0.171040  11.101498  Topic10  -5.3740  -0.9743
14       cloud   0.171195  12.582645  Topic10  -5.3731  -1.0986
17    customer   0.173901  68.993576  Topic10  -5.3575  -2.7846
73  specialist   0.171021  12.333719  Topic10  -5.3741  -1.0797

[414 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         2  0.427736      account
0         7  0.213868      account
0         8  0.267335      account
1         6  0.235453  acquisition
1         7  0.235453  acquisition
...     ...       ...          ...
85        1  0.202677         unit
85        3  0.405354         unit
85        4  0.202677         unit
86        4  0.638651         vice
87        2  0.571464      windows

[200 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 10, 9, 8, 4, 1, 7, 3, 6])

In [ ]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [8]:
apple_sw = pd.read_csv('Data/SoftwareEngineer/SoftwareEngineer_Apple.csv')

In [47]:
#df_clean_1 = column_cleaner(customer_success, 'description')
df_clean_1 = column_cleaner(customer_success[customer_success['prev_title_1'].notna()], 'prev_title_1')

In [ ]:
df['level'=='senior']['total_years_of_prior_exp' > 6]['has_masters' == False]